In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import math

# Load YOLOv8 pose model
model = YOLO("yolov8n-pose.pt")

VIDEO_PATH = "data/fall-01-cam0.mp4"
OUTPUT_PATH = "fall_detection_output.mp4"

cap = cv2.VideoCapture(VIDEO_PATH)

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(
    OUTPUT_PATH,
    fourcc,
    20,
    (int(cap.get(3)), int(cap.get(4)))
)

def calculate_angle(p1, p2):
    dx = p2[0] - p1[0]
    dy = p2[1] - p1[1]
    angle = abs(math.degrees(math.atan2(dy, dx)))
    return angle

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)

    for result in results:
        if result.keypoints is None:
            continue

        keypoints = result.keypoints.xy.cpu().numpy()
        boxes = result.boxes.xyxy.cpu().numpy()

        for i, kps in enumerate(keypoints):
            box = boxes[i]
            x1, y1, x2, y2 = map(int, box)

            width = x2 - x1
            height = y2 - y1

            # COCO keypoints
            left_shoulder = kps[5]
            right_shoulder = kps[6]
            left_hip = kps[11]
            right_hip = kps[12]

            # Compute torso center
            shoulder_center = (left_shoulder + right_shoulder) / 2
            hip_center = (left_hip + right_hip) / 2

            angle = calculate_angle(shoulder_center, hip_center)

            # FALL CONDITION
            is_horizontal = angle < 30 or angle > 150
            lying_down = width > height

            fall_detected = is_horizontal and lying_down

            color = (0, 0, 255) if fall_detected else (0, 255, 0)

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

            label = "FALL DETECTED" if fall_detected else "Normal"
            cv2.putText(frame, f"{label} | Angle: {int(angle)}",
                        (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.6,
                        color,
                        2)

    out.write(frame)
    cv2.imshow("Fall Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

print("Processing complete.")



0: 256x640 1 person, 26.5ms
Speed: 0.7ms preprocess, 26.5ms inference, 0.8ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 3 persons, 21.1ms
Speed: 0.5ms preprocess, 21.1ms inference, 0.3ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 2 persons, 25.5ms
Speed: 0.5ms preprocess, 25.5ms inference, 0.4ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 3 persons, 20.8ms
Speed: 0.5ms preprocess, 20.8ms inference, 0.3ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 3 persons, 24.8ms
Speed: 0.5ms preprocess, 24.8ms inference, 0.3ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 3 persons, 23.9ms
Speed: 0.6ms preprocess, 23.9ms inference, 0.3ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 3 persons, 21.1ms
Speed: 0.8ms preprocess, 21.1ms inference, 0.3ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 2 persons, 22.0ms
Speed: 0.5ms preprocess, 22.0ms inference, 0.3ms postprocess per image at shape (1

: 